In [1]:
import pandas as pd
import gc
import numpy as np
import re
import torch
from tqdm import tqdm
import time

# Finding ML/AI related roles from the Kaggle Linkedin job postings dataset

This analytical workflow guides through the optimization, text feature creation, embedding, indexing, and ANN utilization stages, enabling effective exploration and discovery of machine learning roles within the dataset.

# Table of Contents

**1. Data Preparation and Optimization**

**2. Text Feature Creation**

**3. Text Embedding with Pretrained Model**

**4. Indexing for Efficient Retrieval**

**5. ANN Utilization for Role Discovery**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**1. Data Preparation and Optimization:**

The initial step involves preparing and optimizing the dataset for analysis. This also includes cleaning, preprocessing and merging of dataframes, garbage collection and dropping of irrelevant columns and dataframes.


In [200]:
postings = pd.read_csv('/content/drive/MyDrive/Linkedin_job_postings/postings.csv')

companies = pd.read_csv('/content/drive/MyDrive/Linkedin_job_postings/companies/companies.csv')
company_industries = pd.read_csv('/content/drive/MyDrive/Linkedin_job_postings/companies/company_industries.csv')
company_specialities = pd.read_csv('/content/drive/MyDrive/Linkedin_job_postings/companies/company_specialities.csv')

job_industries = pd.read_csv('/content/drive/MyDrive/Linkedin_job_postings/jobs/job_industries.csv')
job_skills = pd.read_csv('/content/drive/MyDrive/Linkedin_job_postings/jobs/job_skills.csv')

industry_mapping = pd.read_csv('/content/drive/MyDrive/Linkedin_job_postings/mappings/industries.csv')
skills_mapping = pd.read_csv('/content/drive/MyDrive/Linkedin_job_postings/mappings/skills.csv')



Optimizing postings dataframe


In [4]:
# Postings contains the information of all job openings, which will need later in this pipeline
postings.head(2)

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,1.715990e+12,NaN,NaN,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,1.715450e+12,NaN,NaN,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY


In [5]:
# Removing irrelevant columns before merging with other dataframes
postings_df = postings[['job_id','company_name','title','description','skills_desc']]

In [6]:
postings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   job_id        123849 non-null  int64 
 1   company_name  122130 non-null  object
 2   title         123849 non-null  object
 3   description   123842 non-null  object
 4   skills_desc   2439 non-null    object
dtypes: int64(1), object(4)
memory usage: 4.7+ MB


In [7]:
postings_df.size

619245

In [8]:
postings_df.job_id = postings_df.job_id.astype('int32')

<ipython-input-8-459bdc5c8273>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  postings_df.job_id = postings_df.job_id.astype('int32')


In [9]:
postings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   job_id        123849 non-null  int32 
 1   company_name  122130 non-null  object
 2   title         123849 non-null  object
 3   description   123842 non-null  object
 4   skills_desc   2439 non-null    object
dtypes: int32(1), object(4)
memory usage: 4.3+ MB


Optimising companies dataframe

In [10]:
companies.head(2)

,company_id,name,description,company_size,state,country,city,zip_code,address,url
0,1009,IBM,"At IBM, we do more than work. We create. We cr...",7.0,NY,US,"Armonk, New York",10504,International Business Machines Corp.,https://www.linkedin.com/company/ibm
1,1016,GE HealthCare,Every day millions of people feel the impact o...,7.0,0,US,Chicago,0,-,https://www.linkedin.com/company/gehealthcare


In [11]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24473 entries, 0 to 24472
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   company_id    24473 non-null  int64  
 1   name          24472 non-null  object 
 2   description   24176 non-null  object 
 3   company_size  21699 non-null  float64
 4   state         24451 non-null  object 
 5   country       24473 non-null  object 
 6   city          24472 non-null  object 
 7   zip_code      24445 non-null  object 
 8   address       24451 non-null  object 
 9   url           24473 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 1.9+ MB


In [12]:
companies = companies[['company_id','name', 'description']]

In [13]:
companies.company_id.max()

103472979

In [14]:
companies.company_id = companies.company_id.astype('int32')

In [15]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24473 entries, 0 to 24472
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   company_id   24473 non-null  int32 
 1   name         24472 non-null  object
 2   description  24176 non-null  object
dtypes: int32(1), object(2)
memory usage: 478.1+ KB


Optimizing company_industries and company_specialities

In [16]:
company_industries.head(2)

,company_id,industry
0,391906,Book and Periodical Publishing
1,22292832,Construction


In [17]:
company_industries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24375 entries, 0 to 24374
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   company_id  24375 non-null  int64 
 1   industry    24375 non-null  object
dtypes: int64(1), object(1)
memory usage: 381.0+ KB


In [18]:
company_industries.company_id.max()

103472979

In [19]:
company_industries.company_id = company_industries.company_id.astype('int32')

In [20]:
company_industries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24375 entries, 0 to 24374
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   company_id  24375 non-null  int32 
 1   industry    24375 non-null  object
dtypes: int32(1), object(1)
memory usage: 285.8+ KB


Merging companies and company_industries

In [21]:
company_industries = companies.merge(company_industries,how='left',on='company_id')

In [22]:
del companies
gc.collect()

122

In [23]:
company_industries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24483 entries, 0 to 24482
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   company_id   24483 non-null  int32 
 1   name         24482 non-null  object
 2   description  24186 non-null  object
 3   industry     24375 non-null  object
dtypes: int32(1), object(3)
memory usage: 669.6+ KB


In [24]:
company_specialities.head(2)

,company_id,speciality
0,22292832,window replacement
1,22292832,patio door replacement


In [25]:
company_specialities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169387 entries, 0 to 169386
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   company_id  169387 non-null  int64 
 1   speciality  169387 non-null  object
dtypes: int64(1), object(1)
memory usage: 2.6+ MB


In [26]:
company_specialities.company_id.max()

103458790

In [27]:
company_specialities.company_id = company_specialities.company_id.astype('int32')

In [28]:
company_specialities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169387 entries, 0 to 169386
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   company_id  169387 non-null  int32 
 1   speciality  169387 non-null  object
dtypes: int32(1), object(1)
memory usage: 1.9+ MB


Merging company_industries and company_specialities

In [29]:
company_merged = company_industries.merge(company_specialities, how= 'left', on='company_id')

In [30]:
del company_industries, company_specialities
gc.collect()

0

In [31]:
company_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176210 entries, 0 to 176209
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   company_id   176210 non-null  int32 
 1   name         176209 non-null  object
 2   description  175907 non-null  object
 3   industry     176102 non-null  object
 4   speciality   169516 non-null  object
dtypes: int32(1), object(4)
memory usage: 6.0+ MB


Optimizing job_industries and job_skills

In [32]:
job_industries.head(2)

,job_id,industry_id
0,3884428798,82
1,3887473071,48


In [33]:
job_industries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164808 entries, 0 to 164807
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   job_id       164808 non-null  int64
 1   industry_id  164808 non-null  int64
dtypes: int64(2)
memory usage: 2.5 MB


In [34]:
job_industries.job_id.max(),job_industries.industry_id.max()

(3906267224, 3253)

In [35]:
job_industries.job_id = job_industries.job_id.astype('int32')
job_industries.industry_id = job_industries.industry_id.astype('int32')

In [36]:
job_skills.head(2)

,job_id,skill_abr
0,3884428798,MRKT
1,3884428798,PR


In [37]:
job_skills.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213768 entries, 0 to 213767
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   job_id     213768 non-null  int64 
 1   skill_abr  213768 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.3+ MB


In [38]:
job_skills.job_id.max()

3906267224

In [39]:
job_skills.job_id = job_skills.job_id.astype('int32')

In [40]:
job_skills.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213768 entries, 0 to 213767
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   job_id     213768 non-null  int32 
 1   skill_abr  213768 non-null  object
dtypes: int32(1), object(1)
memory usage: 2.4+ MB


Merging job_industries and job_skills

In [41]:
#Include only job_ids with skill_description
job_merged = job_industries.merge(job_skills,how='inner',on='job_id')

In [42]:
del job_industries, job_skills
gc.collect()

0

In [43]:
job_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286885 entries, 0 to 286884
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   job_id       286885 non-null  int32 
 1   industry_id  286885 non-null  int32 
 2   skill_abr    286885 non-null  object
dtypes: int32(2), object(1)
memory usage: 4.4+ MB


Optimizing industry_mapping  and skills_mapping dataframes

In [44]:
industry_mapping.head(2)

,industry_id,industry_name
0,1,Defense and Space Manufacturing
1,3,Computer Hardware Manufacturing


In [45]:
industry_mapping.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422 entries, 0 to 421
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   industry_id    422 non-null    int64 
 1   industry_name  388 non-null    object
dtypes: int64(1), object(1)
memory usage: 6.7+ KB


In [46]:
industry_mapping.describe()

,industry_id
count,422.000000
mean,1342.305687
std,1212.022551
min,1.000000
25%,108.250000
50%,1161.500000
75%,2279.500000
max,3253.000000


In [47]:
industry_mapping.industry_id = industry_mapping.industry_id.astype('int32')

In [48]:
industry_mapping.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422 entries, 0 to 421
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   industry_id    422 non-null    int32 
 1   industry_name  388 non-null    object
dtypes: int32(1), object(1)
memory usage: 5.1+ KB


In [49]:
skills_mapping.head(2)

,skill_abr,skill_name
0,ART,Art/Creative
1,DSGN,Design


In [50]:
skills_mapping.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   skill_abr   35 non-null     object
 1   skill_name  35 non-null     object
dtypes: object(2)
memory usage: 688.0+ bytes


In [51]:
job_merged = job_merged.merge(industry_mapping,how='inner',on='industry_id')

In [52]:
job_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286885 entries, 0 to 286884
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   job_id         286885 non-null  int32 
 1   industry_id    286885 non-null  int32 
 2   skill_abr      286885 non-null  object
 3   industry_name  286724 non-null  object
dtypes: int32(2), object(2)
memory usage: 6.6+ MB


In [53]:
postings_df = postings_df.merge(job_merged,how='left',on='job_id')


In [54]:
del job_merged
gc.collect()

0

In [55]:
postings_df = postings_df.drop(['skill_abr','industry_id'],axis=1)

In [56]:
postings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279762 entries, 0 to 279761
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   job_id         279762 non-null  int32 
 1   company_name   277271 non-null  object
 2   title          279762 non-null  object
 3   description    279750 non-null  object
 4   skills_desc    4504 non-null    object
 5   industry_name  276626 non-null  object
dtypes: int32(1), object(5)
memory usage: 11.7+ MB


In [57]:
postings_df = postings_df.drop_duplicates(subset=['job_id'])


In [58]:
gc.collect()

0

In [59]:
company_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176210 entries, 0 to 176209
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   company_id   176210 non-null  int32 
 1   name         176209 non-null  object
 2   description  175907 non-null  object
 3   industry     176102 non-null  object
 4   speciality   169516 non-null  object
dtypes: int32(1), object(4)
memory usage: 6.0+ MB


In [60]:
company_merged.industry = company_merged.industry.replace({'<NA>': float('nan'), pd.NA: float('nan')})
company_merged.name = company_merged.name.replace({'<NA>': float('nan'), pd.NA: float('nan')})

In [61]:
postings_df.head(10)

,job_id,company_name,title,description,skills_desc,industry_name
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,Requirements: \n\nWe are seeking a College or ...,Real Estate
2,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",NaN,NaN
3,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,We are currently accepting resumes for FOH - A...,Restaurants
5,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,This position requires a baseline understandin...,Law Practice
6,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,NaN,Facilities Services
7,91700727,Downtown Raleigh Alliance,Economic Development and Planning Intern,Job summary:The Economic Development & Plannin...,NaN,Non-profit Organization Management
8,103254301,Raw Cereal,Producer,Company DescriptionRaw Cereal is a creative de...,NaN,Design Services
11,112576855,NaN,Building Engineer,Summary: Due to the pending retirement of our ...,NaN,Food and Beverage Services
15,1218575,Children's Nebraska,Respiratory Therapist,"At Children’s, the region’s only full-service ...",• Requires the ability to communicate effectiv...,Hospitals and Health Care
16,2264355,Bay West Church,Worship Leader,It is an exciting time to be a part of our chu...,"Knowledge, Skills and Abilities: 1. Proficient...",Religious Institutions


In [62]:

postings_df.company_name = postings_df.company_name.replace({'<NA>': float('nan'), pd.NA: float('nan')})
postings_df.industry_name = postings_df.industry_name.replace({'<NA>': float('nan'), pd.NA: float('nan')})


In [63]:
postings_df.title= postings_df.title.fillna('')
postings_df.description = postings_df.description.fillna('')
postings_df.skills_desc = postings_df.skills_desc.fillna('')

In [64]:
postings_df['job_features'] = postings_df[['title', 'description', 'skills_desc']].agg('_'.join, axis=1)

In [65]:
postings_df = postings_df[['job_id','company_name','industry_name','job_features']]

In [66]:
postings_df.head()

,job_id,company_name,industry_name,job_features
0,921716,Corcoran Sawyer Smith,Real Estate,Marketing Coordinator_Job descriptionA leading...
2,1829192,NaN,NaN,Mental Health Therapist/Counselor_At Aspen The...
3,10998357,The National Exemplar,Restaurants,Assitant Restaurant Manager_The National Exemp...
5,23221523,"Abrams Fensterman, LLP",Law Practice,Senior Elder Law / Trusts and Estates Associat...
6,35982263,NaN,Facilities Services,Service Technician_Looking for HVAC service t...


In [67]:
company_merged = company_merged.drop(['company_id'],axis=1)

In [68]:
company_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176210 entries, 0 to 176209
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   name         176209 non-null  object
 1   description  175907 non-null  object
 2   industry     176102 non-null  object
 3   speciality   169516 non-null  object
dtypes: object(4)
memory usage: 5.4+ MB


In [69]:
company_merged.description = company_merged.description.fillna('')
company_merged.speciality = company_merged.speciality.fillna('')


company_merged['company_features'] = company_merged[['description', 'speciality']].agg('_'.join, axis=1)

In [70]:
company_merged = company_merged.drop(['description', 'speciality'],axis=1)
company_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176210 entries, 0 to 176209
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   name              176209 non-null  object
 1   industry          176102 non-null  object
 2   company_features  176210 non-null  object
dtypes: object(3)
memory usage: 4.0+ MB


*Merging company and job data using sqlite for better performance*

Preprocessing text based keys before merging dataframes

In [71]:
def preprocess_key(key):
    if pd.isna(key):
        return np.nan
    # Convert to string, strip spaces, convert to lower case, and remove special characters
    key = str(key).strip().lower()
    key = re.sub(r'[^a-zA-Z0-9]', '', key)  # Remove non-alphanumeric characters
    return key


company_merged.name = company_merged.name.apply(preprocess_key)
company_merged.industry = company_merged.industry.apply(preprocess_key)
postings_df.company_name = postings_df.company_name.apply(preprocess_key)
postings_df.industry_name = postings_df.industry_name.apply(preprocess_key)

In [72]:
import sqlite3

In [73]:
conn = sqlite3.connect('job_postings.db')

In [74]:
postings_df.to_sql('jobs', conn, index=False, if_exists='replace')
company_merged.to_sql('companies', conn, index=False, if_exists='replace')

176210

In [75]:
postings_df.info(),company_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 123849 entries, 0 to 279761
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   job_id         123849 non-null  int32 
 1   company_name   122130 non-null  object
 2   industry_name  120792 non-null  object
 3   job_features   123849 non-null  object
dtypes: int32(1), object(3)
memory usage: 4.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176210 entries, 0 to 176209
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   name              176209 non-null  object
 1   industry          176102 non-null  object
 2   company_features  176210 non-null  object
dtypes: object(3)
memory usage: 4.0+ MB


(None, None)

In [76]:
query = '''
SELECT jobs.job_id, jobs.job_features,companies.name, companies.industry,companies.company_features
FROM jobs
LEFT JOIN companies ON jobs.industry_name = companies.industry
and jobs.company_name = companies.name
'''

In [77]:
merged_df = pd.read_sql_query(query, conn)

In [78]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680087 entries, 0 to 680086
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   job_id            680087 non-null  int64 
 1   job_features      680087 non-null  object
 2   name              642594 non-null  object
 3   industry          642594 non-null  object
 4   company_features  642594 non-null  object
dtypes: int64(1), object(4)
memory usage: 25.9+ MB


**2. Creation of Text-Based Features:**

Generate text-based features for each job-id in the dataset. These features  include descriptions, titles, and other relevant textual information associated with each job.


In [79]:
merged_df['company_features'] = merged_df['name'] + merged_df['industry']
merged_df = merged_df.drop(['name','industry'],axis=1)

In [80]:
merged_df['features'] = merged_df['job_features'] + merged_df['company_features']
merged_df = merged_df[['job_id','features']]

In [81]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680087 entries, 0 to 680086
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   job_id    680087 non-null  int64 
 1   features  642594 non-null  object
dtypes: int64(1), object(1)
memory usage: 10.4+ MB


**3. Text Embedding Using Pretrained Model:**

Utilize a pretrained sentence model to embed the text-based features generated in the previous step. This involves converting the textual information into numerical representations (embeddings) that capture semantic information.

In [83]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 8.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [85]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [89]:
merged_df.dropna(subset = ['features'], inplace=True)

In [92]:
def generate_embeddings(text_column):
    return model.encode(text_column.tolist(), batch_size=64, show_progress_bar=True)

merged_df['features'] = merged_df['features'].astype('str')

embeddings = generate_embeddings(merged_df['features'])


Batches:   0%|          | 0/10627 [00:00<?, ?it/s]

In [93]:
embeddings.shape[1]

384

In [94]:

# Convert embeddings to DataFrame for easier handling
embeddings_df = pd.DataFrame(embeddings, columns=[f'embedding_{i}' for i in range(embeddings.shape[1])])


**4. Indexing for Efficient Retrieval:**

Create an index to efficiently retrieve relevant information from the embedded text features. This step involves organizing the embeddings in a structured manner to enable fast search and retrieval using FAISS (Facebook AI Similarity Search).

Initialize a FAISS index for L2 (Euclidean) distance. The IndexFlatL2 class is used here. Convert the embeddings to float32, as required by FAISS, and add them to the index.



In [96]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 15.5 MB/s eta 0:00:00


In [97]:
import faiss


dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

# Convert embeddings to float32, required by FAISS
embeddings = np.array(embeddings, dtype=np.float32)

# Add embeddings to the index
index.add(embeddings)

In [98]:
merged_df = merged_df.drop(['features'],axis=1)

# Concatenate the embeddings with the original DataFrame
merged_df = pd.concat([merged_df, embeddings_df], axis=1)



In [109]:
merged_df.head()

,job_id,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_374,embedding_375,embedding_376,embedding_377,embedding_378,embedding_379,embedding_380,embedding_381,embedding_382,embedding_383
0,921716,-0.028266,-0.071670,-0.029868,0.061323,0.027085,0.020422,0.049223,-0.010509,-0.045516,...,-0.041593,0.050094,0.012395,0.017271,0.045256,0.106442,-0.058614,-0.006961,-0.092021,0.041435
1,921716,-0.028266,-0.071670,-0.029868,0.061323,0.027085,0.020422,0.049223,-0.010509,-0.045516,...,-0.041593,0.050094,0.012395,0.017271,0.045256,0.106442,-0.058614,-0.006961,-0.092021,0.041435
2,1829192,-0.064298,-0.028240,0.045258,-0.035129,-0.005670,-0.068026,0.118114,-0.015423,0.033077,...,0.066508,-0.086586,0.006683,-0.054190,-0.079507,0.074257,0.165269,0.006237,0.016829,-0.030652
3,10998357,-0.001598,-0.054911,0.032345,0.017314,-0.084667,0.017023,-0.025832,0.001622,-0.059018,...,-0.040638,0.063969,0.007023,0.008837,0.111137,0.020843,0.050181,-0.046106,-0.089872,0.035707
4,23221523,-0.011644,-0.011935,-0.039356,-0.037858,-0.104368,0.004627,-0.077624,0.004940,-0.005565,...,-0.056809,0.039540,0.026226,0.054303,0.001175,0.004092,0.142416,-0.068669,0.009223,0.024219


**5. Approximate Nearest Neighbor (ANN) Search:**

Retrieve the sentences corresponding to the most similar embeddings from the DataFrame using FAISS index search.

In [163]:
def find_similar_sentences(sample_text, df, model, index, top_n=30):
    # Compute embedding for the sample text
    sample_embedding = model.encode([sample_text])

    # Convert sample embedding to float32
    sample_embedding = np.array(sample_embedding, dtype=np.float32)

    # Search the FAISS index
    distances, indices = index.search(sample_embedding, top_n)

    # Retrieve the most similar sentences
    similar_sentences = df.iloc[indices[0]]

    return similar_sentences


In [164]:
sample_text = "machine learning engineer"

similar_sentences = find_similar_sentences(sample_text, merged_df, model, index, top_n=10)

similar_sentences = similar_sentences.drop_duplicates()
job_related_mle = similar_sentences['job_id'].to_list()
output = postings_df[postings_df['job_id'].isin(job_related_mle)]

In [165]:
sample_text = "data science"

similar_sentences = find_similar_sentences(sample_text, merged_df, model, index, top_n=10)

similar_sentences = similar_sentences.drop_duplicates()
job_related_ds = similar_sentences['job_id'].to_list()
output = pd.concat([output, postings_df[postings_df['job_id'].isin(job_related_ds)]], ignore_index=True, axis=0)


In [166]:
sample_text = "machine learning"

similar_sentences = find_similar_sentences(sample_text, merged_df, model, index, top_n=10)

similar_sentences = similar_sentences.drop_duplicates()
job_related_ml = similar_sentences['job_id'].to_list()
output = pd.concat([output, postings_df[postings_df['job_id'].isin(job_related_ai)]], ignore_index=True, axis=0)

In [167]:
sample_text = "artificial intelligence"

similar_sentences = find_similar_sentences(sample_text, merged_df, model, index, top_n=10)

similar_sentences = similar_sentences.drop_duplicates()
job_related_ai = similar_sentences['job_id'].to_list()
output = pd.concat([output, postings_df[postings_df['job_id'].isin(job_related_ai)]], ignore_index=True, axis=0)



In [201]:
postings_original = pd.read_csv('/content/drive/MyDrive/Linkedin_job_postings/postings.csv')

In [202]:
postings.job_id = postings.job_id.astype('int32')
indices = postings[postings['job_id'].isin(output['job_id'])].index

In [203]:
postings_original.iloc[indices]

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type
18730,3888944168,Dice,"Director, Generative AI, Platform Agents and T...",Dice is the leading career destination for tec...,NaN,NaN,United States,6849.0,5.0,NaN,...,1.715225e+12,NaN,Director,NaN,1.712633e+12,click.appcast.io,0,PART_TIME,NaN,NaN
25520,3891075442,Data Glacier,Data Science Intern DIN18,The ideal candidate will use their passion for...,NaN,NaN,United States,69739391.0,3.0,NaN,...,1.715455e+12,NaN,NaN,NaN,1.712863e+12,NaN,0,INTERNSHIP,NaN,NaN
27547,3891278079,Data Glacier,Data Science Intern DIN19,The ideal candidate will use their passion for...,NaN,NaN,United States,69739391.0,8.0,NaN,...,1.715483e+12,NaN,NaN,NaN,1.712891e+12,NaN,0,INTERNSHIP,NaN,NaN
31221,3894617510,Booz Allen Hamilton,Autonomous Systems and Artificial Intelligence...,Job Number: R0194801\n\nAutonomous Systems and...,250000.0,YEARLY,"Washington, DC",1403.0,4.0,NaN,...,1.715489e+12,NaN,NaN,NaN,1.712897e+12,careers.boozallen.com,0,FULL_TIME,USD,BASE_SALARY
68594,3902780915,Capital One,Senior Machine Learning Engineer,"West Creek 1 (12071), United States of America...",NaN,NaN,"Richmond, VA",1419.0,19.0,NaN,...,1.715996e+12,NaN,Mid-Senior level,NaN,1.713404e+12,dsp.prng.co,0,FULL_TIME,NaN,NaN
68600,3902780922,Capital One,Senior Machine Learning Engineer,"West Creek 1 (12071), United States of America...",NaN,NaN,"McLean, VA",1419.0,12.0,NaN,...,1.715996e+12,NaN,Mid-Senior level,NaN,1.713404e+12,dsp.prng.co,0,FULL_TIME,NaN,NaN
68601,3902780923,Capital One,Senior Machine Learning Engineer,"West Creek 1 (12071), United States of America...",NaN,NaN,"Plano, TX",1419.0,20.0,NaN,...,1.715996e+12,NaN,Mid-Senior level,NaN,1.713404e+12,dsp.prng.co,0,FULL_TIME,NaN,NaN
68602,3902780924,Capital One,"Senior Machine Learning Engineer (Python, PySp...","Center 1 (19052), United States of America, Mc...",NaN,NaN,"McLean, VA",1419.0,4.0,NaN,...,1.715996e+12,NaN,Mid-Senior level,NaN,1.713404e+12,dsp.prng.co,0,FULL_TIME,NaN,NaN
68627,3902781837,Capital One,"Senior Machine Learning Engineer (Python, PySp...","Center 1 (19052), United States of America, Mc...",NaN,NaN,"New York, NY",1419.0,6.0,NaN,...,1.715996e+12,NaN,Mid-Senior level,NaN,1.713404e+12,dsp.prng.co,0,FULL_TIME,NaN,NaN
78515,3903813687,"Astir IT Solutions, Inc.",Senior Machine Learning Engineer(Only w2),Senior Machine Learning Engineer(Only w2)Locat...,NaN,NaN,"Raleigh, NC",225796.0,6.0,NaN,...,1.716064e+12,NaN,Mid-Senior level,NaN,1.713472e+12,NaN,0,CONTRACT,NaN,NaN
